In [1]:
import nimblend as nd
import numpy as np

In [2]:
# Create two arrays with different coordinates
data1 = np.array([[1, 2], [3, 4]])
coords1 = {'x': ['a', 'b'], 'y': [0, 1]}
arr1 = nd.Array(data1, coords1)

data2 = np.array([[10, 20, 30], [40, 50, 60]])
coords2 = {'x': ['b', 'c'], 'y': [0, 1, 2]}
arr2 = nd.Array(data2, coords2)

# Operation automatically aligns coordinates
result = arr1 + arr2
print(result)

# Create a lazy array with Dask
lazy_arr = nd.Array(data1, coords1, chunks="auto")
print(lazy_arr.is_lazy)  # True

# Save array to Zarr format
nd.to_zarr(result, "result.zarr")

# Load from Zarr
loaded = nd.from_zarr("result.zarr")

Array((3, 3), {'x': (3,), 'y': (3,)})
True


/home/carlos/remote/nimblend/.venv/lib/python3.13/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/carlos/remote/nimblend/.venv/lib/python3.13/site-packages/zarr/core/array.py:3989: UserWarning: The dtype `<U1` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(
/home/carlos/remote/nimblend/.venv/lib/python3.13/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)


In [3]:
arr1

Dimensions,Coordinates
x,['a' 'b']
y,[0 1]


In [4]:
arr2

Dimensions,Coordinates
x,['b' 'c']
y,[0 1 2]


In [5]:
result

Dimensions,Coordinates
x,['a' 'b' 'c']
y,[0 1 2]


In [6]:
import nimblend as nd
import numpy as np

# Create temperature data for multiple stations over time
times = [1,2,3,4,5,6,7,8]
stations = ['A', 'B', 'C', 'D']
temps = np.random.normal(15, 5, size=(len(times), len(stations)))

# Create a NimbleNd array
temp_array = nd.Array(
    temps,
    coords={'time': times, 'station': stations},
    dims=['time', 'station'],
    name='temperature'
)

# Apply a 3-day rolling mean along the time dimension
# First convert to pandas for the operation
temp_series = nd.to_series(temp_array)
rolled = temp_series.groupby('station').rolling(3).mean()
rolled = rolled.droplevel(0)

# Convert back to NimbleNd
smoothed_temps = nd.from_series(rolled)

# Extract data for a specific station
station_a = temp_array[{'station': 'A'}]


In [7]:
temp_series

time  station
1     A          18.847844
      B          13.174778
      C          19.013597
      D          14.751049
2     A          12.285680
      B          14.091904
      C          11.542981
      D          16.968790
3     A          21.381083
      B           5.977173
      C          15.179438
      D          15.375894
4     A           7.715090
      B          11.724534
      C          13.486774
      D          15.410758
5     A          10.834931
      B          13.994332
      C          11.703974
      D          10.893669
6     A          15.311596
      B          16.484664
      C          11.308292
      D          17.373736
7     A          11.302440
      B           8.210230
      C          14.162070
      D          13.968465
8     A          11.111485
      B          13.723645
      C          15.977808
      D          11.627564
Name: temperature, dtype: float64

In [8]:
smoothed_temps

Dimensions,Coordinates
time,[1 2 3 4 5 6 7 8]
station,['A' 'B' 'C' 'D']


In [9]:
station_a

Dimensions,Coordinates
time,[1 2 3 4 5 6 7 8]
station,['A']


In [10]:
import nimblend as nd
import numpy as np
from scipy import ndimage

# Create sample satellite data with multiple bands
lat = np.linspace(30, 35, 500)
lon = np.linspace(-100, -95, 500)
bands = ['red', 'green', 'blue', 'nir']

# Create some sample data - in practice this would be loaded from files
data = np.zeros((len(lat), len(lon), len(bands)))

# Fill with simulated data
for i, band in enumerate(bands):
    base = np.random.normal(0, 1, size=(len(lat), len(lon)))
    # Apply some smoothing to simulate realistic satellite imagery
    data[:, :, i] = ndimage.gaussian_filter(base, sigma=5)
    # Add some "features"
    if band == 'nir':
        # Higher NIR values in some regions (vegetation)
        data[100:300, 200:400, i] += 2

# Create NimbleNd array
satellite_data = nd.Array(
    data,
    coords={'lat': lat, 'lon': lon, 'band': bands},
    dims=['lat', 'lon', 'band'],
    name='satellite_imagery'
)

# Calculate NDVI (Normalized Difference Vegetation Index)
red = satellite_data[{'band': 'red'}]
nir = satellite_data[{'band': 'nir'}]
ndvi = (nir - red) / (nir + red)
ndvi.name = 'ndvi'

# Save the NDVI result
nd.to_zarr(ndvi, 'ndvi_result.zarr')

/home/carlos/remote/nimblend/.venv/lib/python3.13/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/carlos/remote/nimblend/.venv/lib/python3.13/site-packages/zarr/core/array.py:3989: UserWarning: The dtype `<U3` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(


In [11]:
ndvi

Dimensions,Coordinates
band,['nir' 'red']
lat,[30. 30.01002004 30.02004008 30.03006012 30.04008016 30.0501002 30.06012024 30.07014028 30.08016032 30.09018036]...
lon,[-100. -99.98997996 -99.97995992 -99.96993988 -99.95991984 -99.9498998 -99.93987976 -99.92985972 -99.91983968 -99.90981964]...


In [12]:
ndvi

Dimensions,Coordinates
band,['nir' 'red']
lat,[30. 30.01002004 30.02004008 30.03006012 30.04008016 30.0501002 30.06012024 30.07014028 30.08016032 30.09018036]...
lon,[-100. -99.98997996 -99.97995992 -99.96993988 -99.95991984 -99.9498998 -99.93987976 -99.92985972 -99.91983968 -99.90981964]...


In [13]:
import icechunk

In [14]:
storage_config = icechunk.in_memory_storage()

In [15]:
repo = icechunk.Repository.create(storage_config)

In [16]:
session = repo.writable_session("main")

In [17]:
import nimblend as nd
import numpy as np

# Create an array
data = np.random.rand(1000, 1000)
coords = {'x': np.arange(1000), 'y': np.arange(1000)}
dims=['x', 'y']
arr = nd.Array(data, coords, dims, name='large_data')

# Save to IceChunk

nd.to_icechunk(arr, session, group='my_array')

# Use with Dask for efficient distributed computing
lazy_arr = arr.to_dask()

nd.to_icechunk(lazy_arr, session, group='large_data')

/home/carlos/remote/nimblend/.venv/lib/python3.13/site-packages/zarr/core/group.py:1099: UserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(


TypeError: Expected a BytesBytesCodec. Got <class 'str'> instead.

In [ ]:
session.store